In [4]:
from confluent_kafka import Producer 
import socket
import pandas as pd

conf = {'bootstrap.servers': 'localhost:9092',  # Configurar la dirección del servidor Kafka
        'client.id': socket.gethostname()}  # Configurar el ID del cliente como el nombre del host

producer = Producer(conf)  # Crear una instancia de la clase Producer con la configuración proporcionada

def acked(err, msg):
    """
    Función de callback para manejar el resultado de la entrega del mensaje.

    Parameters:
    - err: Objeto de error, None si la entrega fue exitosa.
    - msg: Objeto de mensaje que contiene información sobre el mensaje producido.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))
    producer.poll(1)  # Permitir que el productor realice cualquier trabajo pendiente durante 1 milisegundo

    json = pd.read_csv('productosJD20231215185723.csv')
    for i in range(len(json)):
        producer.produce('project', key='key', value=str(json), callback=acked)



%3|1702665634.216|FAIL|A208-100#producer-4| [thrd:A208-100:9092/0]: A208-100:9092/0: Connect to ipv4#127.0.1.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%6|1702665654.923|FAIL|A208-100#producer-4| [thrd:A208-100:9093/1]: A208-100:9093/1: Disconnected (after 19705ms in state UP)
%3|1702665756.216|FAIL|A208-100#producer-4| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1702665761.216|FAIL|A208-100#producer-4| [thrd:A208-100:9092/0]: A208-100:9092/0: Connect to ipv4#127.0.1.1:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1702665762.217|FAIL|A208-100#producer-4| [thrd:A208-100:9093/1]: A208-100:9093/1: Connect to ipv4#127.0.1.1:9093 failed: Connection refused (after 0ms in state CONNECT)
%3|1702665767.216|FAIL|A208-100#producer-4| [thrd:A208-100:9093/1]: A208-100:9093/1: Connect to ipv4#127.0.1.1:9093 failed: Connection 

In [ ]:
from confluent_kafka import Producer 
import socket
import json

conf = {'bootstrap.servers': 'localhost:9092',  # Configurar la dirección del servidor Kafka
        'client.id': socket.gethostname()}  # Configurar el ID del cliente como el nombre del host

producer = Producer(conf)  # Crear una instancia de la clase Producer con la configuración proporcionada

def acked(err, msg):
    """
    Función de callback para manejar el resultado de la entrega del mensaje.

    Parameters:
    - err: Objeto de error, None si la entrega fue exitosa.
    - msg: Objeto de mensaje que contiene información sobre el mensaje producido.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))
    producer.poll(1)  # Permitir que el productor realice cualquier trabajo pendiente durante 1 milisegundo

# Cargar el contenido del archivo JSON
with open('productosJD20231215185723.json', 'r') as file:
    json_content = json.load(file)

# Enviar cada elemento del JSON como un mensaje Kafka
for item in json_content:
    # Convierte el diccionario a una cadena JSON antes de enviarlo
    json_string = json.dumps(item)
    producer.produce('project', key="key", value=json_string, callback=acked)

# Esperar a que todos los mensajes se envíen antes de salir
producer.flush()
